In [3]:
import os
import os.path
authors = [
    "Topsoee", "JacobsenJP", "SkramE", "SkramA", "AndersenHC",
    "Bauditz", "Bang", "Kielland", "Schandorph", "Drachman",
    "Hamsun", "HansenJH", "Pontoppidan", "GB",
    "Levison", "Fibiger"
    ]
paths = [os.path.join("korpus", filename) for filename in os.listdir("korpus")
         if filename.endswith(".txt") and
         any((("_" + author + "_") in filename) for author in authors)]
len(paths)

def split(list_a, chunk_size):
    for i in range(0, len(list_a), chunk_size):
        yield list_a[i:i + chunk_size]
        
docs = []
for path in paths:
    with open(path, encoding="utf-8") as f:  # 75 token chunks
        docs += split(f.read().split(), 75)
docs[:3]
len(docs)

60373

In [7]:
!mkdir -p wsd/test-da
from xml.sax.saxutils import escape
with open("wsd/test-da/test-da.data.xml", "w", encoding="utf-8") as f, open("wsd/test-da/test-da.gold.key.txt", "w", encoding="utf-8") as g:
    print('<?xml version="1.0" encoding="UTF-8"?>', file=f)
    print('<corpus lang="da" name="test-da" sources="wordnet-danish">', file=f)
    print('   <text id="d000">', file=f)
    for i, doc in enumerate(docs):
        print('      <sentence id="s{:03d}" source="wordnet-danish">'.format(i), file=f)
        for token in doc:
            if token == "skæbne":
                print('         <instance id="s{:03d}.t000">{}</instance>'.format(i), file=f)
                print('s{:03d}.t000 bn:00026603n', file=g)
            else:
                print('         <wf>{}</wf>'.format(escape(token)), file=f)
        print('      </sentence>', file=f)
    print('    </text>', file=f)
    print('</corpus>', file=f)

In [31]:
from pandas import read_csv
import os
import os.path

data = []
for filename in os.listdir("annotations"):
    path = os.path.join("annotations", filename)
    if path.endswith(".csv"):
        df = read_csv(path, delimiter=";")
        data += df[~df['match'].isnull()].to_dict(orient="records")
data[1]

{'corpus': 'MeMo corpus v. 0.5',
 'match_position': 22977994.0,
 'left context': 'Enten—Eller , enten Liv eller Død — saa levende var den almindelige Interesse for og Deltagelse i den dømte Forbryders',
 'match': 'Skæbne',
 'right_context': '. Pludselig gjorde Bødlen , søm hidtil havde staaet ubevægelig og med korslagte Arme , . en Bevægelse . Han',
 'Kodning': 0.0,
 'text_gender': nan,
 'corpus_id': 'MEMO_ALL',
 'text_nationality': nan,
 'text_illustrations': 'n',
 'text_subtitle': 'Original Roman fra Nutiden',
 'text_publisher': 'Simonsen & Co.',
 'text_id': '1880_RH_ArbejderBankierOgBaronEllerLykkensOmskiftelser',
 'text_pseudonym': 'R.H.',
 'text_pages': 750.0,
 'text_source': 'KB',
 'text_surname': nan,
 'text_file_id': 130024104401.0,
 'text_year': 1880.0,
 'text_typeface': 'gothic',
 'text_volume': nan,
 'text_price': '5',
 'text_title': 'Arbejder, Bankier og Baron eller Lykkens Omskiftelser',
 'text_firstname': nan}

In [35]:
from xml.sax.saxutils import escape

with open("wsd/test-da/test-da.data.xml", "w", encoding="utf-8") as f, open("wsd/test-da/test-da.gold.key.txt", "w", encoding="utf-8") as g:
    print('<?xml version="1.0" encoding="UTF-8"?>', file=f)
    print('<corpus lang="da" name="test-da" sources="wordnet-danish">', file=f)
    print('   <text id="d000">', file=f)
    for i, line in enumerate(data):
        print('      <sentence id="d000.s{:03d}" source="wordnet-danish">'.format(i), file=f)
        for token in line["left context"].split():
            print('         <wf lemma="" pos="">{}</wf>'.format(escape(token)), file=f)
        print('         <instance id="d000.s{:03d}.t000" lemma="" pos="">{}</instance>'.format(i, escape(line["match"])), file=f)
        print('d000.s{:03d}.t000 {}'.format(i, 'bn:00026603n' if int(line["Kodning"]) == 0 else 'bn:00019222n'), file=g)
        for token in line["right_context"].split():
            print('         <wf lemma="" pos="">{}</wf>'.format(escape(token)), file=f)
        print('      </sentence>', file=f)
    print('    </text>', file=f)
    print('</corpus>', file=f)

In [ ]:
!PYTHONPATH=. python src/evaluation/evaluate_model.py --config /work/config_da.test.yaml --checkpoint_path pretrained_models/bert-base-multilingual-cased/model_state_best.th  --cpu